Visualize the scalability as a function of dimension in the linear case

In [1]:
import numpy as np
%matplotlib inline  
import glob,h5py,os,re
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2.5
mpl.rcParams['lines.markersize']=8
mpl.rcParams['text.usetex']=True
mpl.rcParams['text.latex.unicode']=True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['text.latex.preamble']= ['\usepackage{amsfonts}','\usepackage{amsmath}']
mpl.rcParams['font.size'] = 20
mpl.rcParams['axes.labelsize']=20
mpl.rcParams['legend.fontsize']=20
#http://stackoverflow.com/questions/22408237/named-colors-in-matplotlib
import cPickle as pickle
from datasets.synthp import params_synthetic
from utils.misc import getPYDIR,getConfigFile,readPickle, loadHDF5

#visualize synthetic results

results = {}
results[10] = {}
results[10]['name']     = 'synthetic12'
results[10]['maxEPOCH'] = 200
results[10]['dim']      = 10
results[100] = {}
results[100]['name']     = 'synthetic13'
results[100]['maxEPOCH'] = 200
results[100]['dim']      = 100

ntrainlist = [10,100,1000]
SAVEDIR = '../../expt-synthetic-fast/chkpt-'

DATADIR = getPYDIR()+'/datasets/synthetic/'

baselines = {}
for dset in results:
    bline = '../../expt-synthetic-fast/baselines/'+results[dset]['name']+'-baseline.h5'
    if os.path.exists(bline):
        bb = loadHDF5(bline)
        print bb.keys()
    else:
        bb = None
    baselines[dset] = bb


from datasets.load import loadDataset
datasets = {}
dset_params = {}
for dset in results: 
    datasets[dset]    = loadDataset(results[dset]['name'])
    dset_params[dset] = params_synthetic[results[dset]['name']]

Loading linear matrices
[u'test_rmse', u'valid_mu', u'test_cov', u'train_cov', u'valid_rmse', u'test_ll', u'train_rmse', u'valid_cov', u'train_mu', u'train_ll', u'test_mu', u'valid_ll']
[u'test_rmse', u'valid_mu', u'test_cov', u'train_cov', u'valid_rmse', u'test_ll', u'train_rmse', u'valid_cov', u'train_mu', u'train_ll', u'test_mu', u'valid_ll']
Found:  /data/ml2/rahul/theanomodels/datasets/synthetic//synthetic12.h5
Found:  /data/ml2/rahul/theanomodels/datasets/synthetic//synthetic13.h5


In [23]:
#Visualizing epoch
getepoch = re.compile("-EP(.*)-")

def estimateMSE(mu_posterior, true_z):
    err_sum = np.square(mu_posterior-true_z).sum()
    rmse = np.sqrt(err_sum/float(np.prod(mu_posterior.shape))).squeeze()
    #assert rmse.shape[0]==mu_posterior.shape[2],'Shape not matched'
    return rmse

plt.figure(figsize = (8,10))
for dset in results:
    DIR = SAVEDIR+results[dset]['name']+'/'
    for f in glob.glob(DIR+'*-final.h5'):
        params  = readPickle(getConfigFile(f))[0]
        Ntrain, dimstoc =  params['ntrain'],results[dset]['dim']
        alldata = loadHDF5(f)
        print f,alldata.keys()
        valid_mus= alldata['mu_posterior_valid']
        valid_zs = datasets[dimstoc]['valid_z']
        print Ntrain,dimstoc,  valid_mus.shape, valid_zs.shape
        #rmse_train = estimateMSE(valid_mus,valid_zs)
        print rmse_train

../../expt-synthetic-fast/chkpt-synthetic12/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-40-ds-100-nl-relu-bs-250-ep-200-rs-40-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-100-uid-final.h5 [u'valid_bound', u'train_bound', u'mu_posterior_train', u'mu_posterior_valid', u'valid_nll', u'cov_posterior_valid', u'cov_posterior_train', u'tsbn_bound']
100 10 (100, 25, 10) (500, 25, 10)
3.14621497768
../../expt-synthetic-fast/chkpt-synthetic12/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-40-ds-100-nl-relu-bs-250-ep-200-rs-40-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-10-uid-final.h5 [u'valid_bound', u'train_bound', u'mu_posterior_train', u'mu_posterior_valid', u'valid_nll', u'cov_posterior_valid', u'cov_posterior_train', u'tsbn_bound']
10 10 (10, 25, 10) (500, 25, 10)
3.14621497768
../../expt-synthetic-fast/chkpt-synthetic12/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-40-ds-100-nl-relu-bs-250-ep-200-rs-40-ttype-simple_gated-e

In [ ]:
getepoch = re.compile("-EP(.*)-")
paramconfig =''

markers  = {}
markers['ST-L']  = 'v'
markers['ST-LR'] = '<'
markers['ST-R']  = '>'
markers['MF-L']  = '8'
markers['MF-LR'] = '1'
markers['MF-R']  = 'p'
markers['x']  = 's'
markers['z']  = 's'
markers['KF'] = 's'

colors   = {}
colors['ST-L']  = 'k'
colors['ST-LR'] = 'b'
colors['ST-R']  = 'g'
colors['MF-L']  = 'y'
colors['MF-LR'] = 'r'
colors['MF-R']  = 'c'
colors['x']  = '#FF6347'
colors['z']  = '#FF6347'
colors['KF'] = '#C0C0C0'

def estimateMSE(mu_posterior, true_z):
    err_sum = np.square(mu_posterior-true_z.repeat(mu_posterior.shape[2],2)).sum(0,keepdims=True).sum(1,keepdims=True)
    rmse = np.sqrt(err_sum/float(mu_posterior.shape[0]*mu_posterior.shape[1])).squeeze()
    assert rmse.shape[0]==mu_posterior.shape[2],'Shape not matched'
    return rmse

#Train and validate bounds
fig_train,axlist_train = plt.subplots(1,2,figsize = (12,4))
fig_valid,axlist_valid = plt.subplots(1,2,figsize = (12,4))
for f in glob.glob(SAVEDIR+'/*'+paramconfig+'*-final.h5'):
    code    = getCode(f)
    print f
    synthData= getHDF5Data(DATADIR+'/'+dataset+'.h5')
    if code in ['MF-R']:
        continue
    alldata = getHDF5Data(f)
    config  = getPKLData(f.replace('final','config').replace('h5','pkl'))
    X_Y     = alldata['train_bound'][10:,:][::config['savefreq'],:]
    print X_Y[:,0],maxEPOCH
    idxMax  = np.where(X_Y[:,0]>maxEPOCH)[0][0]
    plotBounds(axlist_train.ravel()[1],X_Y[:idxMax,0],X_Y[:idxMax,1],colors[code], markers[code],label = code)
    X_Y     = alldata['valid_bound'][1:,:]
    idxMax  = np.where(X_Y[:,0]>maxEPOCH)[0][0]
    plotBounds(axlist_valid.ravel()[1],X_Y[:idxMax,0],X_Y[:idxMax,1],colors[code], markers[code],label = code)
    print alldata['mu_posterior_train'].shape,synthData['train_z'].shape
    
    rmse_train = estimateMSE(alldata['mu_posterior_train'],synthData['train_z'])
    rmse_valid = estimateMSE(alldata['mu_posterior_valid'],synthData['valid_z'])
    X          = np.arange(len(rmse_train))*config['savefreq']
    idxMax  = np.where(X>maxEPOCH)[0][0]
    plotBounds(axlist_train.ravel()[0],X[:idxMax],rmse_train[:idxMax], colors[code], markers[code],label = code)
    plotBounds(axlist_valid.ravel()[0],X[:idxMax],rmse_valid[:idxMax], colors[code], markers[code],label = code)

def plotHorizontal(ax, value, l, m, c):
    L = int(ax.get_xlim()[1])
    X = np.arange(0,maxEPOCH,100)
    ax.plot(X,np.array([value]*len(X)),label=l, marker =m, color=c)
    
#Add in plots for KF 
if params_synthetic[dataset]['baseline']=='KF':
    bline_train_ll = -baseline['train_ll'][0]/dset['mask_train'].sum()
    plotHorizontal(axlist_train.ravel()[1], bline_train_ll, 'KF', markers['KF'], colors['KF'])
    bline_valid_ll = -baseline['valid_ll'][0]/dset['mask_valid'].sum()
    plotHorizontal(axlist_valid.ravel()[1], bline_valid_ll, 'KF', markers['KF'], colors['KF'])

ax = axlist_train.ravel()[0]
bline_train_rmse = baseline['train_rmse']
baseline_label = params_synthetic[dataset]['baseline']
if baseline_label=='KF':
    baseline_label+=' [Exact]'
plotHorizontal(axlist_train.ravel()[0], bline_train_rmse, baseline_label, 
               markers['KF'], colors['KF'])

ax = axlist_valid.ravel()[0]
bline_valid_rmse = baseline['valid_rmse']
plotHorizontal(axlist_valid.ravel()[0], bline_valid_rmse, baseline_label, 
               markers['KF'], colors['KF'])

print params_synthetic[dataset].keys()
ax = axlist_train.ravel()[0]
ax.set_xlabel('Epochs')
ax.set_ylabel('Train RMSE')
ax.set_ylim([1,6])
#ax.set_title(params_synthetic[dataset]['docstr'])
ax = axlist_train.ravel()[1]
if dataset!='synthetic9':
    ax.set_title(params_synthetic[dataset]['docstr'])
ax.set_xlabel('Epochs')
ax.set_ylabel('Train Upper Bound')
ax = axlist_valid.ravel()[0]
ax.set_xlabel('Epochs')
ax.set_ylabel('Validate RMSE')
ax.set_ylim([1,6])
#ax.set_title(params_synthetic[dataset]['docstr'])
ax = axlist_valid.ravel()[1]
ax.set_xlabel('Epochs')
ax.set_ylabel('Validate Upper Bound')
ax.set_title(params_synthetic[dataset]['docstr'])

if dataset=='synthetic9':
    axlist_train[0].legend(loc='upper center', bbox_to_anchor=(0.5, 0.5),ncol=2, frameon=False)
elif dataset=='synthetic10':
    axlist_train[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.),ncol=2, frameon=False)
else:
    axlist_train[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.4),ncol=3, frameon=False)
plt.figure(fig_train.number)
plt.savefig(dataset+'-train-bound.pdf',bbox_inches='tight')
if dataset=='synthetic9':
    axlist_valid[0].legend(loc='upper center', bbox_to_anchor=(0.5, 0.5),ncol=2, frameon=False)
elif dataset=='synthetic10':
    axlist_valid[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.),ncol=2, frameon=False)
else:
    axlist_valid[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.4),ncol=3, frameon=False)
plt.figure(fig_valid.number)
plt.savefig(dataset+'-valid-bound.pdf',bbox_inches='tight')

In [ ]:
#Reconstructions
MARKER  = -1
nplots  = 2

#Train Reconstructions
fig,axlist  = plt.subplots(nplots,2,figsize = (16,8))
#if dataset!='synthetic9':
    #plt.suptitle(params_synthetic[dataset]['docstr'],fontsize=28,y=1.02)
fig.subplots_adjust(hspace=.3)
np.random.seed(3)
idxlist     = np.random.permutation(500)[:nplots]
print idxlist

def plotRecons(axlist, data, idxlist, c,m,label = None):
    for idx,n in enumerate(idxlist):
        Y = data[n,:]
        X = np.arange(len(Y))
        axlist[idx].plot(X,Y,color=c,marker=m,label=label)

def plotMuCov(axlist, data, stdData, idxlist, c,m,label = None):
    for idx,n in enumerate(idxlist):
        Y = data[n,:]
        X = np.arange(len(Y))
        std = stdData[n,:]
        axlist[idx].plot(X,Y,color=c,marker=m,label=label)
        axlist[idx].fill_between(X, Y+std, Y-std, facecolor=c, alpha=0.5)
        
for fnum,f in enumerate(glob.glob(SAVEDIR+'/*'+paramconfig+'*-final.h5')):
    synthData= getHDF5Data(DATADIR+'/'+dataset+'.h5')
    code    = getCode(f)
    if fnum==0:
        #Plot the original data
        mus_method = synthData['train_z'].squeeze()
        x_method = synthData['train'].squeeze()
        plotRecons(axlist[:,0], mus_method, idxlist, colors['z'],markers['z'],label='z')
        plotRecons(axlist[:,1], x_method, idxlist, colors['x'],markers['x'],label='x')
        #Plot baseline
        mus_method = baseline['train_mu'].squeeze()
        std_method = np.sqrt(baseline['train_cov'].squeeze())
        plotMuCov(axlist[:,0], mus_method,std_method, idxlist, colors['KF'],markers['KF'],
                  label=params_synthetic[dataset]['baseline'])
        
    if code not in ['ST-R']:
        continue
    alldata = getHDF5Data(f)
    mus_method = alldata['mu_posterior_train'][:,:,MARKER].squeeze()
    std_method = np.sqrt(alldata['cov_posterior_train'][:,:,MARKER])
    x_method   = params_synthetic[dataset]['obs_fxn'](mus_method)
    std_x_method= np.ones_like(x_method)*np.sqrt(params_synthetic[dataset]['obs_cov'])
    
    plotMuCov(axlist[:,0], mus_method,std_method, idxlist, colors[code],markers[code],label=code)
    plotMuCov(axlist[:,1], x_method,std_x_method, idxlist, colors[code],markers[code],label=code)

#Also add exact inference to the plot
ax = axlist[0,0]
ax.set_title('Latent Space')
ax = axlist[1,0]
yspace = 1.2
ax.legend(loc='upper center', bbox_to_anchor=(0.5, yspace),ncol=3, frameon=False)

ax = axlist[0,1]
ax.set_title('Observations')
ax = axlist[1,1]
ax.legend(loc='upper center', bbox_to_anchor=(0.5, yspace),ncol=3, frameon=False)
plt.savefig(dataset+'-vis.pdf',bbox_inches='tight')